# PRODUCTS

**A. CLEAN UP AND PREPARE PRODUCT DATA**

A.1 Load and plot

In [ ]:
%matplotlib inline
import pandas as pd

import matplotlib.pyplot as plt

import os

In [ ]:
os.getcwd()

In [ ]:
#load the data
hh_demographic = pd.read_csv('../data/dunnhumby_complete_csv/hh_demographic.csv', sep = ',')

transaction_data = pd.read_csv('../data/dunnhumby_complete_csv/transaction_data.csv', sep = ',')

product = pd.read_csv('../data/dunnhumby_complete_csv/product.csv', sep = ',')

In [ ]:
product.head(4)

We reorder the manufacturer and department order :

In [ ]:
ordered_manufacturer= product.MANUFACTURER.astype(int).sort_values()
ordered_manufacturer= ordered_manufacturer.unique()

product['MANUFACTURER'] = pd.Categorical(product['MANUFACTURER'],
                      ordered = True,
                      categories = ordered_manufacturer)

print ('The order of the manufacturer categories is :', ordered_manufacturer)

In [ ]:
ordered_department= ['GROCERY','FROZEN GROCERY','GRO BAKERY','PASTRY',
        'SEAFOOD','SEAFOOD-PCKGD','MEAT', 'MEAT-PCKGD','MEAT-WHSE','PORK','SPIRITS','NUTRITION',
       'PRODUCE','CHEF SHOPPE','RESTAURANT','SALAD BAR','DELI', 'DAIRY DELI','DELI/SNACK BAR',
    'DRUG GM','COSMETICS','PHARMACY SUPPLY','GARDEN CENTER','FLORAL', 'TRAVEL & LEISUR', 
    'POSTAL CENTER', 'PHOTO', 'VIDEO','VIDEO RENTAL','TOYS','HOUSEWARES', 
    'KIOSK-GAS', 'ELECT &PLUMBING','AUTOMOTIVE','GM MERCH EXP','COUP/STR & MFG', 
     'CHARITABLE CONT', 'RX','PROD-WHS SALES', 'HBC','CNTRL/STORE SUP','MISC. TRANS.', 'MISC SALES TRAN',' ']

product['DEPARTMENT'] = pd.Categorical(product['DEPARTMENT'],
                      ordered = True,
                      categories = ordered_department)

print ('The order of the department categories is :', ordered_department)

In [ ]:
department= product.groupby(['DEPARTMENT']).count()

fig = plt.figure(figsize=(20,10))

# Divide the figure into a 2x1 grid, and give me the first section
ax1 = fig.add_subplot(231)

# Divide the figure into a 2x1 grid, and give me the second section
ax2 = fig.add_subplot(232)

ax3 = fig.add_subplot(233)

ax4 = fig.add_subplot(234)

ax5 = fig.add_subplot(235)

ax6 = fig.add_subplot(236)

department[(department.BRAND>0) & (department.BRAND<=2)].BRAND.sort_values().plot.bar(ax=ax1)
department[(department.BRAND>2) & (department.BRAND<=5)].BRAND.sort_values().plot.bar(ax=ax2)
department[(department.BRAND>5) & (department.BRAND<=20)].BRAND.sort_values().plot.bar(ax=ax3)
department[(department.BRAND>20) & (department.BRAND<=500)].BRAND.sort_values().plot.bar(ax=ax4)
department[(department.BRAND>500) & (department.BRAND<=4000)].BRAND.sort_values().plot.bar(ax=ax5)
department[department.BRAND>4000].BRAND.sort_values().plot.bar(ax=ax6)

fig.subplots_adjust(hspace=1)
fig.subplots_adjust(wspace=0.4)


**We look at the product data and try to understand what they stand for:<br/>**
 1) MEAT-PCKGD and SEAFOOD-PCKGD are prepared meat or seafood meals.<br/>
 2) DRUG GM : the overall transactions from the small entrance DRUGSTORE when going to the supermarket.<br/>
 3) GROCERY : the overall product bought in the grocery store, mostly food and some hygien and cleaning products.<br/> 

In [ ]:
# we try to understand what MEAT-PCKGD and SEAFOOD-PCKGD are : we can see that it is prepared meat or seafood meals.
product[product['DEPARTMENT'] == 'MEAT-PCKGD'].SUB_COMMODITY_DESC.unique()
product[product['DEPARTMENT'] == 'SEAFOOD-PCKGD'].SUB_COMMODITY_DESC.unique()

# we try to understand what DRUG MG stands for : unfortunatly we can see that it contains products from a lot 
# of different categories like : food, hygien product, stationery products, toys etc. What I understand from it 
#is that it corresponds to the overall transactions from the small entrance DRUGSTORE when going to the supermarket 
product[product['DEPARTMENT'] == 'DRUG GM'].SUB_COMMODITY_DESC.unique()

product[product['DEPARTMENT'] == 'GROCERY'].COMMODITY_DESC.unique()


In [ ]:
#we extract the grocery product data which is the main supermarket in the commercial center
Grocery= pd.DataFrame(product[product['DEPARTMENT'] == 'GROCERY'])
Grocery.head()

In [ ]:
split = Grocery['COMMODITY_DESC'].str.split(' |\/').tolist()
words = pd.DataFrame.from_records(split, index= Grocery.index, columns=["w1","w2","w3","w4","w5","w6"])
words.groupby('w1').count()

In [ ]:
words.w1.unique()

In [ ]:
FoodW1 = ['FRZN', 'FRUIT', 'COOKIES', 'SPICES','CHEESE','COFFEE','BAKED', 'VEGETABLES','CONDIMENTS', 'BAKING',
    'PICKLE','SOUP','FROZEN', 'EGGS','REFRGRATD', 'HOT', 'COLD', 'SUGARS', 'SEAFOOD','CANNED','BEANS', 'CRACKERS',
     'BUTTER', 'WATER','MILK','TEAS', 'PASTA', 'MOLASSES','MEAT', 'SALD', 'LIQUOR', 'SNACK', 'BEERS', 'YOGURT',
    'COCOA', 'FLOUR', 'SYRUPS','OLIVES','ISOTONIC']
        
OtherW1 = ['ICE','PNT','AIR','SHORTENING','PAPER','HISPANIC','DINNER','HOUSEHOLD','FD','DRY','FLUID','BAG',
         'CONVENIENT', 'SOFT','WAREHOUSE','PWDR','MISC.','BLEACH','DOMESTIC',
       'MISC', 'COUPON', 'IMPORTED', 'RESTRICTED', 'NEW','BOTTLE']        

PetW1 = ['CAT','DOG','PET','BIRD']

SanitaryW1 = ['FACIAL','LAUNDRY','BATH','DISHWASH']


In [ ]:
words.w2.unique()

In [ ]:
FoodW2 = ['BREAD','SAUCES', 'VEGETABLE','FRUITS', 'WRAPS','SWEET','RELISH','PIZZA', 'DOUGH', 'CEREAL', 
        'SWEETNERS', 'JUICES', 'SNACKS','BRKFST', 'DRINKS', 'MEAT','NOODLES', 'SAUCE', 'FOOD', 'SYRUP','PIE',
        'NUTS', 'ALES','FOODS','BREAKFAST','DAIRY','TOPPINGS','POTATOES','CHICKEN', 'WINE','DIET', 'SEED', 
        'SEAFOOD']

OtherW2 =['ICE', '-', 'CONES', '&', 'BTR', 'CREAM','OIL','HOUSEWARES','MXS:DRY','NEEDS','MIX', 
          'MIXES','BN', 'TISS','MISC','BY-PRODUCTS','JCE', 'ADDITIVES','DRSNG','NOVELTIES', 'CRYSTL', 'AGE', 
          'DEPOSITS']

SanitaryW2=['CARE','CLEANG','DETERGENTS','TISSUES','TOWELS','LITTER']

In [ ]:
words.w3.unique()

In [ ]:
FoodW3 = ['JELLY', 'MILK', 'BUNS', 'VEG','DESSERTS', 'CANNED','MEAT','FOODS','PASTA',
       'PANCAKE', 'SNDWCH', 'DOUGH', 'DRNKS', 'WTR', 'GRAVY', 'DRNK',
       'MEALS']

OtherW3 =['SHELF', 'EXTRACTS','NEEDS',
       'BAGS','PKLD', 'PRODUCTS', 'GOODS', 'DNR','BKD', 'WHLSM','CARBONATED','CONC','SUPPLIES', 'ITEMS']

In [ ]:
Grocery['IsFood'] = False
Grocery['IsPet'] = False
Grocery['IsSanitary'] = False
Grocery['IsOther'] = False

In [ ]:
Grocery.head(4)

In [ ]:
words['IsFoodW1'] = words.w1.isin(FoodW1)
words['IsSanitaryW1'] = words.w1.isin(SanitaryW1)
words['IsPetW1'] = words.w1.isin(PetW1)
words['IsOtherW1'] = words.w1.isin(OtherW1)

words['IsFoodW2'] = words.w2.isin(FoodW2)
words['IsSanitaryW2'] = words.w2.isin(SanitaryW2)
words['IsOtherW2'] = words.w2.isin(OtherW2)

words['IsFoodW3'] = words.w3.isin(FoodW3)
words['IsOtherW3'] = words.w3.isin(OtherW3)


In [ ]:
words.head()

In [ ]:
# a loop to check if we have non concordance. Everything seems fine. With looking at the three first words 
# we can determine the type of product : food, pet, sanitary or other.

#uncomment to check :

#for i in words.index : 
#    if (words['IsOtherW1'].loc[i]) and (words['IsOtherW2'].loc[i]) and (words['IsOtherW3'].loc[i]) :
#        print(words.loc[i])

In [ ]:
#now we need to fill the Grocery table with the type of elements :

for i in Grocery.index : 
    if (words['IsOtherW1'].loc[i]) and (words['IsOtherW2'].loc[i]) and (words['IsOtherW3'].loc[i]) :
        Grocery.IsFood.loc[i] = False
        Grocery.IsPet.loc[i] = False
        Grocery.IsSanitary.loc[i] = False
        Grocery.IsOther.loc[i] = True
        
    elif (words['IsPetW1'].loc[i]) :
        Grocery.IsFood.loc[i] = False
        Grocery.IsPet.loc[i] = True
        Grocery.IsSanitary.loc[i] = False
        Grocery.IsOther.loc[i] = False
    
    elif (words['IsFoodW1'].loc[i]) or (words['IsFoodW2'].loc[i]) or (words['IsFoodW3'].loc[i]) :
        Grocery.IsFood.loc[i] = True
        Grocery.IsPet.loc[i] = False
        Grocery.IsSanitary.loc[i] = False
        Grocery.IsOther.loc[i] = False
        
    elif (words['IsSanitaryW1'].loc[i]) or (words['IsSanitaryW2'].loc[i]):
        Grocery.IsFood.loc[i] = False
        Grocery.IsPet.loc[i] = False
        Grocery.IsSanitary.loc[i] = True
        Grocery.IsOther.loc[i] = False

In [ ]:
Grocery.head(30)

In [ ]:
Grocery.groupby('IsFood').count()